In [ ]:
import numpy as np
import axc_opt

obj = axc_opt.ObjectiveFunction()
objfunc = obj.get_function()

In [ ]:
objfunc([3, 0.05, 0.05])

In [ ]:
# objfunc([5, 0.9])
# [objfunc([x, 0.95]) for x in [3, 4, 5, 5.5]]
[objfunc([x, 0.05, 0.05]) for x in [3, 4, 5, 6]]

In [ ]:
from noisyopt import minimizeCompass

res = minimizeCompass(
    objfunc,
    x0=np.array([5, 0.05, 0.05]),
    scaling=[1, 100, 100],
    bounds=[[3, 6], [0.0, 0.3], [0.0, 5.0]],
    errorcontrol=False,
)
res

In [ ]:
from noisyopt import minimizeSPSA

res = minimizeSPSA(
    objfunc,
    x0=np.array([5, 0.05, 0.05]),
    bounds=[[3, 6], [0.00, 0.3], [0.0, 5.0]],
    niter=5000,
)
res